# Neural Network Training

## Multi-Layer Perceptron

In [12]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report

# 1. Data Loading and Preparation
data = pd.read_csv('emotion_data.csv')
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

X = data[feature_cols]
y = data[target_col]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 2. Define the Neural Network Model using PyTorch
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes)
        )
        
    def forward(self, x):
        return self.model(x)

input_dim = X_train_tensor.shape[1]
num_classes = len(np.unique(y_encoded))
model = MLP(input_dim, num_classes)

# 3. Define Loss, Optimizer, and Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# 4. Training the Model with Epoch-by-Epoch Accuracy Monitoring
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    train_loss = running_loss / total_train
    train_acc = correct_train / total_train
    
    # Evaluate test accuracy
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
    test_acc = correct_test / total_test
    
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Test Acc: {test_acc:.4f}")

# 5. Final Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

print(f"\nFinal Test Accuracy: {np.mean(np.array(all_preds) == y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, all_preds, target_names=label_encoder.classes_))

Epoch 1/50 - Loss: 0.9407 - Train Acc: 0.6992 - Test Acc: 0.7100
Epoch 2/50 - Loss: 0.8763 - Train Acc: 0.7089 - Test Acc: 0.7100
Epoch 3/50 - Loss: 0.8669 - Train Acc: 0.7090 - Test Acc: 0.7100
Epoch 4/50 - Loss: 0.8614 - Train Acc: 0.7091 - Test Acc: 0.7100
Epoch 5/50 - Loss: 0.8553 - Train Acc: 0.7085 - Test Acc: 0.7100
Epoch 6/50 - Loss: 0.8554 - Train Acc: 0.7092 - Test Acc: 0.7100
Epoch 7/50 - Loss: 0.8521 - Train Acc: 0.7086 - Test Acc: 0.7100
Epoch 8/50 - Loss: 0.8513 - Train Acc: 0.7088 - Test Acc: 0.7101
Epoch 9/50 - Loss: 0.8480 - Train Acc: 0.7090 - Test Acc: 0.7100
Epoch 10/50 - Loss: 0.8459 - Train Acc: 0.7082 - Test Acc: 0.7099
Epoch 11/50 - Loss: 0.8443 - Train Acc: 0.7088 - Test Acc: 0.7100
Epoch 12/50 - Loss: 0.8442 - Train Acc: 0.7082 - Test Acc: 0.7100
Epoch 13/50 - Loss: 0.8442 - Train Acc: 0.7087 - Test Acc: 0.7100
Epoch 14/50 - Loss: 0.8436 - Train Acc: 0.7093 - Test Acc: 0.7078
Epoch 15/50 - Loss: 0.8434 - Train Acc: 0.7086 - Test Acc: 0.7100
Epoch 16/50 - Loss:

C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report

# 1. Data Loading and Preparation
# Load your dataset from the CSV file
data = pd.read_csv('emotion_data.csv')

# Define the target variable and feature columns
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

# Select features (X) and target (y)
X = data[feature_cols]
y = data[target_col]

# Print the first few rows to verify the data
print("Selected Features:")
print(X.head())
print("\nTarget Variable:")
print(y.head())

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Splitting and Scaling the Data
# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize the StandardScaler and scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Define the Updated Neural Network Model using PyTorch
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        return self.model(x)

input_dim = X_train_tensor.shape[1]
num_classes = len(np.unique(y_encoded))
model = MLP(input_dim, num_classes)

# 4. Define Loss, Optimizer, and Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Lower learning rate for finer convergence
num_epochs = 50

# 5. Training the Model with Epoch-by-Epoch Monitoring
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    train_loss = running_loss / total_train
    train_acc = correct_train / total_train
    
    # Evaluate test accuracy
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
    test_acc = correct_test / total_test
    
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Test Acc: {test_acc:.4f}")

# 6. Final Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

print(f"\nFinal Test Accuracy: {np.mean(np.array(all_preds) == y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, all_preds, target_names=label_encoder.classes_))

Selected Features:
   sentiment_score  text_length  word_count  avg_word_length  stopword_count  \
0           0.0000           10           3                3               1   
1          -0.7269           61          10                6               3   
2          -0.7351           75          14                5               5   
3          -0.4215           59          11                5               3   
4          -0.4939           66          14                4               8   

   first_person_pronoun_count  keyword_count  
0                           1              0  
1                           0              0  
2                           0              0  
3                           1              0  
4                           0              0  

Target Variable:
0     surprise
1         fear
2         fear
3    Uncertain
4         fear
Name: predicted_emotion, dtype: object
Epoch 1/50 - Loss: 0.9041 - Train Acc: 0.7057 - Test Acc: 0.7100
Epoch 2/50 - Loss: 0.

C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report

# 1. Data Loading and Preparation
data = pd.read_csv('emotion_data.csv')
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

X = data[feature_cols]
y = data[target_col]

print("Selected Features:")
print(X.head())
print("\nTarget Variable:")
print(y.head())

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Splitting and Scaling the Data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Define the Updated Neural Network Model with Batch Normalization
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        return self.model(x)

input_dim = X_train_tensor.shape[1]
num_classes = len(np.unique(y_encoded))
model = MLP(input_dim, num_classes)

# 4. Define Loss, Optimizer, and Learning Rate Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Lower learning rate for finer convergence

# Scheduler to reduce LR when training loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

num_epochs = 100

# 5. Training the Model with Metrics Printed Every 10 Epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Evaluate on test set
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
    test_loss /= total_test
    test_acc = correct_test / total_test

    # Step the scheduler with the training loss
    scheduler.step(train_loss)

    # Print metrics every 10 epochs (and also for the first epoch)
    if (epoch+1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} - Test Acc: {test_acc:.4f}")

# 6. Final Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

print(f"\nFinal Test Accuracy: {np.mean(np.array(all_preds) == y_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, all_preds, target_names=label_encoder.classes_))

Selected Features:
   sentiment_score  text_length  word_count  avg_word_length  stopword_count  \
0           0.0000           10           3                3               1   
1          -0.7269           61          10                6               3   
2          -0.7351           75          14                5               5   
3          -0.4215           59          11                5               3   
4          -0.4939           66          14                4               8   

   first_person_pronoun_count  keyword_count  
0                           1              0  
1                           0              0  
2                           0              0  
3                           1              0  
4                           0              0  

Target Variable:
0     surprise
1         fear
2         fear
3    Uncertain
4         fear
Name: predicted_emotion, dtype: object


C:\Users\annas\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100 - Train Loss: 0.9189 - Train Acc: 0.6983 - Test Loss: 0.8435 - Test Acc: 0.7101
Epoch 10/100 - Train Loss: 0.8394 - Train Acc: 0.7091 - Test Loss: 0.8366 - Test Acc: 0.7096
Epoch 20/100 - Train Loss: 0.8346 - Train Acc: 0.7092 - Test Loss: 0.8327 - Test Acc: 0.7103
Epoch 30/100 - Train Loss: 0.8301 - Train Acc: 0.7090 - Test Loss: 0.8373 - Test Acc: 0.7100
Epoch 40/100 - Train Loss: 0.8294 - Train Acc: 0.7091 - Test Loss: 0.8350 - Test Acc: 0.7105
Epoch 50/100 - Train Loss: 0.8268 - Train Acc: 0.7091 - Test Loss: 0.8364 - Test Acc: 0.7101
Epoch 60/100 - Train Loss: 0.8244 - Train Acc: 0.7090 - Test Loss: 0.8362 - Test Acc: 0.7096
Epoch 70/100 - Train Loss: 0.8197 - Train Acc: 0.7098 - Test Loss: 0.8354 - Test Acc: 0.7094
Epoch 80/100 - Train Loss: 0.8196 - Train Acc: 0.7099 - Test Loss: 0.8380 - Test Acc: 0.7093
Epoch 90/100 - Train Loss: 0.8175 - Train Acc: 0.7097 - Test Loss: 0.8373 - Test Acc: 0.7090
Epoch 100/100 - Train Loss: 0.8154 - Train Acc: 0.7090 - Test Loss: 0.8

C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Convolutional Neural Network

In [25]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report

# 1. Data Loading and Preparation
data = pd.read_csv('emotion_data.csv')
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

X = data[feature_cols]
y = data[target_col]

print("Selected Features:")
print(X.head())
print("\nTarget Variable:")
print(y.head())

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Splitting and Scaling the Data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Define the CNN Model for Classification
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNClassifier, self).__init__()
        # The input will be reshaped to (batch_size, 1, input_dim)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=2)
        # Compute the output dimension after conv/pooling:
        # If input_dim = 7, then:
        # After conv1: 7 - 2 + 1 = 6, then pooling: floor(6/2)=3
        # After conv2: 3 - 2 + 1 = 2. 
        # Flattened dimension = 64 * 2 = 128.
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        # Reshape input from (batch_size, input_dim) to (batch_size, 1, input_dim)
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc(x)
        return x

input_dim = X_train_tensor.shape[1]  # should be 7
num_classes = len(np.unique(y_encoded))
model = CNNClassifier(input_dim, num_classes)

# 4. Define Loss, Optimizer, and Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# 5. Training the CNN Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Evaluate on test set every 10 epochs
    if (epoch + 1) % 10 == 0:
        model.eval()
        running_loss_test = 0.0
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss_test += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_test += (preds == labels).sum().item()
                total_test += labels.size(0)
        test_loss = running_loss_test / total_test
        test_acc = correct_test / total_test
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} - Test Acc: {test_acc:.4f}")

# 6. Final Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

final_accuracy = np.mean(np.array(all_preds) == y_test)
print(f"\nFinal Test Accuracy: {final_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, all_preds, target_names=label_encoder.classes_))

Selected Features:
   sentiment_score  text_length  word_count  avg_word_length  stopword_count  \
0           0.0000           10           3                3               1   
1          -0.7269           61          10                6               3   
2          -0.7351           75          14                5               5   
3          -0.4215           59          11                5               3   
4          -0.4939           66          14                4               8   

   first_person_pronoun_count  keyword_count  
0                           1              0  
1                           0              0  
2                           0              0  
3                           1              0  
4                           0              0  

Target Variable:
0     surprise
1         fear
2         fear
3    Uncertain
4         fear
Name: predicted_emotion, dtype: object
Epoch 10/50 - Train Loss: 0.8313 - Train Acc: 0.7087 - Test Loss: 0.8380 - Test Acc: 0

C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
# Tuning the CNN

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report

# 1. Data Loading and Preparation
data = pd.read_csv('emotion_data.csv')
target_col = 'predicted_emotion'
feature_cols = [
    'sentiment_score', 'text_length', 'word_count', 
    'avg_word_length', 'stopword_count', 
    'first_person_pronoun_count', 'keyword_count'
]

X = data[feature_cols]
y = data[target_col]

print("Selected Features:")
print(X.head())
print("\nTarget Variable:")
print(y.head())

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Splitting and Scaling the Data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader objects for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Define the Tuned CNN Model for Classification
class TunedCNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TunedCNNClassifier, self).__init__()
        # Reshape input (batch_size, 1, input_dim)
        # First convolutional block
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        # Second convolutional block
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout_conv = nn.Dropout(0.3)
        # Fully connected layers: Calculate flattened size:
        # For input_dim = 7:
        # After conv1: 7 - 2 + 1 = 6, then pooling: floor(6/2)=3.
        # After conv2: 3 - 2 + 1 = 2, so flattened size = 128 * 2 = 256.
        self.fc1 = nn.Linear(256, 128)
        self.dropout_fc = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = x.unsqueeze(1)  # Reshape to (batch_size, 1, input_dim)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout_conv(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout_fc(x)
        x = self.fc2(x)
        return x

input_dim = X_train_tensor.shape[1]  # 7 features
num_classes = len(np.unique(y_encoded))
model = TunedCNNClassifier(input_dim, num_classes)

# 4. Define Loss, Optimizer, and Learning Rate Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
num_epochs = 50

# 5. Training the Tuned CNN Model with Metrics Printed Every 10 Epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Evaluate on test set
    model.eval()
    running_loss_test = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss_test += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
    test_loss = running_loss_test / total_test
    test_acc = correct_test / total_test

    scheduler.step(train_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} - Test Acc: {test_acc:.4f}")

# 6. Final Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

final_accuracy = np.mean(np.array(all_preds) == y_test)
print(f"\nFinal Test Accuracy: {final_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, all_preds, target_names=label_encoder.classes_))

Selected Features:
   sentiment_score  text_length  word_count  avg_word_length  stopword_count  \
0           0.0000           10           3                3               1   
1          -0.7269           61          10                6               3   
2          -0.7351           75          14                5               5   
3          -0.4215           59          11                5               3   
4          -0.4939           66          14                4               8   

   first_person_pronoun_count  keyword_count  
0                           1              0  
1                           0              0  
2                           0              0  
3                           1              0  
4                           0              0  

Target Variable:
0     surprise
1         fear
2         fear
3    Uncertain
4         fear
Name: predicted_emotion, dtype: object


C:\Users\annas\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 10/50 - Train Loss: 0.8445 - Train Acc: 0.7086 - Test Loss: 0.8369 - Test Acc: 0.7104
Epoch 20/50 - Train Loss: 0.8372 - Train Acc: 0.7087 - Test Loss: 0.8410 - Test Acc: 0.7105
Epoch 30/50 - Train Loss: 0.8308 - Train Acc: 0.7089 - Test Loss: 0.8345 - Test Acc: 0.7102
Epoch 40/50 - Train Loss: 0.8280 - Train Acc: 0.7088 - Test Loss: 0.8341 - Test Acc: 0.7100
Epoch 50/50 - Train Loss: 0.8238 - Train Acc: 0.7098 - Test Loss: 0.8345 - Test Acc: 0.7099

Final Test Accuracy: 0.7099

Classification Report:
              precision    recall  f1-score   support

   Uncertain       0.71      1.00      0.83      6553
       anger       0.00      0.00      0.00       167
     disgust       0.00      0.00      0.00         9
        fear       0.00      0.00      0.00       668
         joy       0.00      0.00      0.00       232
     neutral       0.00      0.00      0.00        12
     sadness       0.45      0.01      0.02      1520
    surprise       0.00      0.00      0.00        69


C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\annas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Comparison Summary

An average F1-score of 0.71 suggests the models capture relevant patterns but may fall short for applications requiring higher precision and recall. This plateau indicates potential gaps in the feature set or modeling approach, as traditional models and neural networks show consistent but limited performance. The complexity of mental health classification may require more advanced text representations or contextual features. Modest improvements across iterations suggest challenges like class overlap and data quality. To enhance performance, exploring ensemble methods, refining feature engineering, and integrating domain-specific knowledge could be beneficial.